In [1]:
from Bio import Entrez, SeqIO
Entrez.email = 'curiousgeorge@ufl.edu'

#prompt user for organism and number of results desired
query= input('Enter organism name')
usermax=input('Enter maximum number of results ')

#send query to Entrez nucleotide database
handle = Entrez.esearch(db='nucleotide', term = query, field='organism', retmax=usermax)
record = Entrez.read(handle)
handle.close()

#get the 'IDList' field from the results
ids=(record["IdList"])

#function to get sequences from the genbank IDs returned from the search and write to 'query_fastas.txt'
def get_sequences():
    open('query.fasta','w')
    for seq_id in ids:
        handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text",retmax=1)
        local_file=open('query.fasta','a') #a = append mode
        local_file.write(handle.read()) #add data to end of file
get_sequences()
handle.close()

Enter organism name morchella
Enter maximum number of results  30


In [3]:
#PART 2 - parsing the metadata from the sequences returned from our search, printing matching records... 
my_seqlist = []
for seq_record in SeqIO.parse("query.fasta", "fasta"):
    my_seqlist.append(seq_record)
    my_seqlist[0]
gene=input("Enter Gene of Interest: ")
with open("second.fasta", "w") as handle:
    for seq_record in my_seqlist:
        if gene in seq_record.description:
            SeqIO.write(seq_record, handle, "fasta")

Enter Gene of Interest:  ITS


In [4]:
#Code adapted from Biopython help forums and adapted from work in the Biopython Cookbook Change et al 2020

from Bio import SeqIO
from Bio.SeqUtils.CheckSum import seguid

def remove_duplicates(records):
    check_sequences = set()
    for record in records:
        checksum = seguid(record.seq)
        if checksum in check_sequences:
            print("Removing duplicates %s into outfile" % record.id ) 
            continue
        check_sequences.add(checksum)
        yield record
        if record.id == record.seq:
            print("No duplicates!")

saved_sequences = remove_duplicates(SeqIO.parse("second.fasta", "fasta"))
count = SeqIO.write(saved_sequences, "final.fasta", "fasta")
print ("Number of remaining sequences %s" %count)

#read the new file and display the gene ids

print("Unique designations are: ")

for seq_record in SeqIO.parse("final.fasta", "fasta"):
    print(seq_record.id)

Number of remaining sequence 2
Unique designations are: 
NR_168766.1
NR_168765.1
